# Temperature-Dependent Power Flow

We implement Temperature-Dependent Power Flow (TDPF) in pandapower based on the following publications:

S. Frank, J. Sexauer and S. Mohagheghi, "Temperature-dependent power flow", IEEE Transactions on Power Systems, vol. 28, no. 4, pp. 4007-4018, Nov 2013.

B. Ngoko, H. Sugihara and T. Funaki, "A Temperature Dependent Power Flow Model Considering Overhead Transmission Line Conductor Thermal Inertia Characteristics," 2019 IEEE International Conference on Environment and Electrical Engineering and 2019 IEEE Industrial and Commercial Power Systems Europe (EEEIC / I&CPS Europe), 2019, pp. 1-6, doi: 10.1109/EEEIC.2019.8783234.

This tutorial demonstrates how to use this feature in pandapower in order to calculate temperature of overhead lines and also obtain the results of the TDPF with the adjusted line parameters. Furthermore, thermal inertia of overhead lines can be considered by specifying the time delay after which the temperature should be calculated. This implementation also supports the distributed slack power flow calculation.

We use the approach from Frank et al. to calculate the thermal resistance of the lines and to define the Jacobian matrix and the mismatch eqiation. The approach of Ngoko et al. is used to update the thermal resistance based on the line current and the weather parameters, and to introduce the thermal delay term in the Jacobian matrix. This allows using both methods. Furthermore, the approach to use thermal resistance is more versatile because it allows expanding the implementation for underground cables and transformers in the future.

This implementation will be of benefit for grid studies of Dynamic Line Rating (DLR). In addition, a calculation of a grid state at a specified time after an N-1 event provides additional capabilities for contingency studies and for obtaining emergency line ratings for a given grid. 

In this tutorial, we demonstrate the use of this feature with a small example.

After importing the necessary libraries, we create a 5-bus power system (inspired by Ngoko et al.) that we use to demonstrate the calculation:

In [ ]:
import numpy as np
import pandas as pd
from pandapower.test.loadflow.test_tdpf import simple_test_grid
from pandapower.pf.create_jacobian_tdpf import calc_T_ngoko, calc_a0_a1_a2_tau, calc_T_frank, calc_g_b, calc_i_square_p_loss, calc_r_theta, calc_r_theta_from_t_rise
from pandapower.pypower.idx_brch import BR_R, BR_X
from pandapower.plotting.simple_plot import simple_plot
from pandapower.run import runpp

In [ ]:
net = simple_test_grid(load_scaling=0.25, sgen_scaling=0.5, with_gen=True, distributed_slack=False)

In [ ]:
simple_plot(net, plot_loads=True, plot_sgens=True, plot_gens=True, load_size=3, sgen_size=3, gen_size=3)

In [ ]:
runpp(net)
net.res_line

In [ ]:
net.res_bus

## Calculation of the overhead line temperature

It is possible to use the functions from the implementation separately, in order to calculate the overhead line temperature.

The implementation in Ngoko et al. defines three terms to represent a simplified thermal model of an overhead line. The parameters $a_0$, $a_1$, and $a_2$ specify a constant, linear, and quadratic terms of the dependence of the line temperature from the square of the current $I$ that is flowing through the overhead line:

$$
T_{ss} = a_0 + a_1 \cdot I^2 + a_2 \cdot I^4
$$

$$
T = T_{ss} - (T_{ss} - T_0) \cdot \exp(-t/\tau)
$$

In the equations above, $T_{ss}$ stands for the steady-state temperature, $T_0$ is the initial temperature, and $T$ stands for the temperature at the time $t$ after a step change of current. The parameter $\tau$ is a time constant, used in the caluclation of line temperature with thermal inertia. After $\tau$ seconds, the temperature reaches approximately 63.2 % of the steady-state value.

This approach requires the following inputs (default values are in parentheses):

* air temperature (20 °C)
* initial line temperature (20 °C)
* reference temperature (20 °C)
* resistance at reference temperature (r_ohm_per_km)
* conductor outer diameter
* specific mass of the conductor multiplied by the specific thermal capacity of the material
* wind speed (0.6 m/s)
* wind angle of attack (45 °)
* solar radiation (900 W/m²)
* solar emissivity and solar absorptivity (0.5)
* thermal coefficient of resistivity (4.03e-3)

The parameters are specified in the net.line element table. If any parameters are missing, they will be substitutet by default assumptions. The parameter "tdpf" is always required. If the option to not update $R_\Theta$ is used, then "r_theta_kelvin_per_mw" is required. Otherwise, "conductor_outer_diameter_m" is required. Furthermore, if thermal inertia is considered by setting the option "tdpf_delay_s", the parameter "mc_joule_per_m_k" is required.

In [ ]:
t_air_pu = 35
alpha_pu = 4.03e-3
r_ref = net.line.r_ohm_per_km.values / 1e3
a0, a1, a2, tau = calc_a0_a1_a2_tau(t_air_pu, 80, 20, r_ref, 30.6e-3,
                                    1490, 0.6, 45, 900, alpha_pu, 0.5, 0.5)
calc_T_ngoko(np.square(net.res_line.i_ka.values * 1e3), a0, a1, a2, None, None, None)

In the approach of Frank et al., the parameters specifying the thermal model according to Ngoko et al. are used to calculate the thermal resistamce $R_{\Theta}$: 

$$
R_{\Theta} = \frac{T_{Rise}}{P_{Loss}} = \frac{a_0 - T_{air} + a_1 \cdot I^2 + a_2 \cdot I^4}{P_{Loss}}
$$

$$
T = T_{air} + R_\Theta \cdot P_{Loss}
$$

Alternatively, the thermal resistance can be approximated using an assumption for the rated temperature rise, as described in Frank et al. To this end, the air temperature $T_{air}$ and the temperature rise are added to calculate the line temperature. The temperature rise is calculated by multiplying the thermal resistance $R_\Theta$ and the active power losses $P_{Loss}$. The thermal resistance is included in the line results table.

In [ ]:
branch = net._ppc["branch"]
tdpf_lines = np.ones(len(branch)).astype(bool)
r = branch[tdpf_lines, BR_R].real
x = branch[tdpf_lines, BR_X].real
g, b = calc_g_b(r, x)
Vm = abs(net._ppc["internal"]["V"])
Va = np.angle(net._ppc["internal"]["V"])
i_square_pu, p_loss_pu = calc_i_square_p_loss(branch, tdpf_lines, g, b, Vm, Va)
r_theta_pu = calc_r_theta(t_air_pu, a0, a1, a2, np.square(net.res_line.i_ka.values*1e3), p_loss_pu)
calc_T_frank(p_loss_pu, t_air_pu, r_theta_pu, None, None, None)

In [ ]:
# using an approximation for the rated temperature rise:
r_theta_kelvin_per_mw = calc_r_theta_from_t_rise(net, 25)
calc_T_frank(p_loss_pu, t_air_pu, r_theta_kelvin_per_mw.values * net.sn_mva / 1, None, None, None)

By providing the option tdpf=True, the overhead line temperature can be obtained from the power flow caluclation:

In [ ]:
runpp(net, tdpf=True, max_iteration=30)

Taking the results of the power flow calculation for the line current and using them as the input for the introduced functions, we can make sure that the temperature calculation delivers the same results. However, if we repeat the power flow calculation without the TDPF, the results will be different because the effect of the temperature on the resitance and the current is ignored.

In [ ]:
net.res_line

In [ ]:
calc_T_ngoko(np.square(net.res_line.i_ka.values * 1e3), a0, a1, a2, None, None, None)

In [ ]:
runpp(net)
calc_T_ngoko(np.square(net.res_line.i_ka.values * 1e3), a0, a1, a2, None, None, None)

The calculation works for distributed slack power flow, too. However, the parameter net.sn_mva should be increased in order to avoid numerical instability.

In [ ]:
net.sn_mva = 1000
runpp(net, tdpf=True, tdpf_delay_s=5 * 60, distributed_slack=True, max_iteration=30)

Finally, we demonstrate the calculation with a time delay of 5 minutes:

In [ ]:
runpp(net, tdpf=True, tdpf_delay_s=5 * 60, max_iteration=30)

In [ ]:
net.res_line

We can see that the temperature rise is substantially lower due to thermal inertia. The thermal inertia effect can be illustrated in the following figure.

In [ ]:
delays = np.arange(0, 65, 5)
delay_tab = pd.DataFrame(index=delays, columns=net.line.index.values)

for d in delays:
    runpp(net, tdpf=True, tdpf_delay_s=d * 60, max_iteration=30)
    delay_tab.loc[d, :] = net.res_line.temperature_degree_celsius.values
    
delay_tab.plot(ylabel="Temperature (°C)", xlabel="Time delay (min)", 
               title="Time delay and overhead line temperature");